In [2]:
!pip install pypdf2
!pip install pdf2image
!pip install pytesseract
!pip install tqdm
!pip install tabula-py
!apt install ghostscript python3-tk
!pip install opencv-python
!pip install pdfplumber
!pip install --upgrade pymupdf
!pip install spacy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 53.0 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
python3-tk is already the newest version (3.10.8-1~22.04).
The following additional packages will be installed:
  fonts-droid-fallback fonts-noto-mono fonts-urw-base35 libgs9 libgs9-common libidn12 libijs-0.35
  libjbig2dec0 poppler-data
Suggested packages:
  fonts-noto fonts-freefont-otf | fonts-freefont-ttf fonts-texgyre ghostscript-x poppler-utils
  fonts-japanese-mincho | fonts-ipafont-mincho fonts-japanese-gothic | fonts-ipafont-gothic
  fonts-arphic-ukai fonts-arphic-uming fonts-nanum
The following NEW packages will be installed:
  fonts-droid-fallback fonts-noto-mono fonts-urw-base35 ghostscript libgs9 libgs9-common libidn12
  libijs-0.35 libjbig2dec0 poppler-data
0 upgraded, 10 newly installed, 0 to remove and 49 not upgraded.

In [3]:
from PyPDF2 import PdfReader
from pdf2image import convert_from_path
import pytesseract
import os
import json
from PIL import Image
import re
from tqdm import tqdm
from IPython.display import display
import pandas as pd
import shutil

In [4]:
# Load the PDF
pdf_path = 'Understanding-NEPA-Litigation_v4.pdf'
reader = PdfReader(pdf_path)

In [5]:
print("no. of pages = ", len(reader.pages))

no. of pages =  20


In [6]:
raw_text = ""
page_text = {}
c = 0
for page in tqdm(reader.pages):
    raw_text += page.extract_text() + "\n ---------- NEW PAGE ---------- \n"
    page_text[c+1] = page.extract_text()
    c+=1

100%|██████████| 20/20 [00:00<00:00, 26.87it/s]


### Just check the content

In [7]:
print(page_text[3])

3
Key findings:  
•  Between 2013 and 2022, circuit courts heard approximately 39 NEPA appeals cases per year,  
a 56% increase over the rate from 2001 to 2015 .2
•  Agencies won about 80% of the 2013-2022 appeals cases, 11% more per year than from 2001 
to 2004 , 8% more than from 2001 to 2008 , and 4% less than from 2009 to 2015 .3 The rate at 
which agencies’ reviews are upheld is high, meaning these environmental reviews are seldom 
changed as a result of litigation.
•  On average, 4.2 years elapsed between publication of an environmental impact statement  
or environmental assessment and conclusion of the corresponding legal challenge at the 
appellate level. Of these appealed cases, 84% were closed less than six years after the contested 
permit was published, and 39% were closed in less than three.
•  Among the challenges, 42% contested environmental impact statements, and 36% contested 
environmental assessments. Agencies won about 80% of challenges to both.
•  NGOs instigated 

In [8]:
cleaned_text = re.sub(r'[^a-zA-Z0-9\s.,!?@#$%^&*()+=:;\'"-]', '', raw_text)

In [9]:
clean_pagewise_text = {}
for i in page_text:
    clean_pagewise_text[i] = re.sub(r'[^a-zA-Z0-9\s.,!?@#$%^&*()+=:;\'"-]', '', page_text[i])

In [10]:
print(clean_pagewise_text[3])

3
Key findings:  
  Between 2013 and 2022, circuit courts heard approximately 39 NEPA appeals cases per year,  
a 56% increase over the rate from 2001 to 2015 .2
  Agencies won about 80% of the 2013-2022 appeals cases, 11% more per year than from 2001 
to 2004 , 8% more than from 2001 to 2008 , and 4% less than from 2009 to 2015 .3 The rate at 
which agencies reviews are upheld is high, meaning these environmental reviews are seldom 
changed as a result of litigation.
  On average, 4.2 years elapsed between publication of an environmental impact statement  
or environmental assessment and conclusion of the corresponding legal challenge at the 
appellate level. Of these appealed cases, 84% were closed less than six years after the contested 
permit was published, and 39% were closed in less than three.
  Among the challenges, 42% contested environmental impact statements, and 36% contested 
environmental assessments. Agencies won about 80% of challenges to both.
  NGOs instigated 72% of

### NER with basic spacy model

In [11]:
import spacy

# Load SpaCy model
nlp = spacy.load("en_core_web_sm")

docs = []

for i in range(len(clean_pagewise_text)):
    doc = nlp(clean_pagewise_text[i+1])
    docs.append(doc)

In [12]:
from spacy import displacy


displacy.render(docs[2], style="ent", jupyter=True)


In [13]:
displacy.render(docs[3], style="ent", jupyter=True)

### NER with transformer models

In [14]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")

nlp = pipeline("ner", model=model, tokenizer=tokenizer)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/59.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/829 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/433M [00:00<?, ?B/s]

Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


In [15]:
ner_results = nlp(clean_pagewise_text[3])
print(ner_results)

[{'entity': 'B-ORG', 'score': 0.9645595, 'index': 16, 'word': 'NE', 'start': 81, 'end': 83}, {'entity': 'I-ORG', 'score': 0.974242, 'index': 17, 'word': '##PA', 'start': 83, 'end': 85}, {'entity': 'B-ORG', 'score': 0.97522664, 'index': 249, 'word': 'NE', 'start': 1184, 'end': 1186}, {'entity': 'I-ORG', 'score': 0.9703287, 'index': 250, 'word': '##PA', 'start': 1186, 'end': 1188}]
